In [10]:
import omnia_ae.http_client

ModuleNotFoundError: No module named 'omnia_ae'

In [14]:
import os.path
import sys
sys.path.append(os.path.join(os.path.dirname(__file__), '..'))

NameError: name '__file__' is not defined

In [11]:

#from omnia_ae.http_client import HttpClient, ContentType, RequestType
#from omnia_ae.models import (
#    SourceModel
#)
#from omnia_ae.api import AEAPI, AEEnvironment
from typing import List, Literal, Optional
from azure.identity._internal.msal_credentials import MsalCredential

In [13]:
from .omnia_ae.api import AEAPI, AEEnvironment

ImportError: attempted relative import with no known parent package

In [2]:
from typing import Literal, Optional, TypedDict, Union, Dict, Any
from azure.identity._internal.msal_credentials import MsalCredential
import requests
import logging

#from omnia_timeseries.helpers import retry
#from omnia_timeseries.models import TimeseriesRequestFailedException
from importlib import metadata
from opentelemetry.instrumentation.requests import RequestsInstrumentor
import platform

In [3]:
from azure.identity import ClientSecretCredential
import os
credential = ClientSecretCredential(
    tenant_id=os.environ['AZURE_TENANT_ID'],
    client_id=os.environ['AZURE_CLIENT_ID_NON_PRODUCTION'],
    client_secret=os.environ['AZURE_CLIENT_SECRET_NON_PRODUCTION']
)

In [4]:
base_url = "https://api-dev.gateway.equinor.com/iiot/ae/v1"

In [5]:
ContentType = Literal["application/json",
                      "application/protobuf", "application/x-google-protobuf"]

RequestType = Literal['get']
def _request(
    request_type: RequestType,
    url: str,
    headers: Dict[str, Any],
    payload: Optional[Union[TypedDict, dict, list]] = None,
    params: Optional[Dict[str, Any]] = None
) -> Union[Dict[str, Any], bytes]:

    response = requests.request(
        request_type, url, headers=headers, json=payload, params=params)
    #if not response.ok:
    #    raise TimeseriesRequestFailedException(response)
    if not "Accept" in headers or headers["Accept"] == "application/json":
        return response.json()
    else:
        return response.content

class HttpClient:
    def __init__(self, azure_credential: MsalCredential, resource_id: str):
        self._azure_credential = azure_credential
        self._resource_id = resource_id

    def request(
        self,
        request_type: RequestType,
        url: str,
        accept: ContentType = "application/json",
        payload: Optional[Union[TypedDict, dict, list]] = None,
        params: Optional[Dict[str, Any]] = None
    ) -> Any:

        access_token = self._azure_credential.get_token(
            f'{self._resource_id}/.default')  # handles caching and refreshing internally
        headers = {
            'Authorization': f'Bearer {access_token.token}',
            'Content-Type': 'application/json',
            'Accept': accept,
            ##'User-Agent': f'Omnia AE SDK/{version} {system_version_string}'
        }
        return _request(request_type=request_type, url=url, headers=headers, payload=payload, params=params)


In [6]:
#resource_id="32f2a909-8a98-4eb8-b22d-1208d9350cb0"
resource_id="657b2767-ee47-47ff-b745-af501ea053cb"

In [7]:
print(base_url)

https://api-dev.gateway.equinor.com/iiot/ae/v1


In [8]:
system_version_string = f'({platform.system()}; Python {platform.version()})' if platform.system(
) else f'(Python {platform.version()})'

In [9]:
access_token = credential.get_token(
    f'{resource_id}/.default')  # handles caching and refreshing internally
headers = {
    'Authorization': f'Bearer {access_token.token}',
    'Content-Type': 'application/json',
    'Accept': 'application/json',
    'User-Agent': f'Omnia AE SDK/{1.0} {system_version_string}'
        }

In [15]:
#access_token.token

In [11]:
_request(request_type='get', url=base_url+'/sources', headers=headers, payload=None, params=None)

{'items': [{'facility': 'AHA'},
  {'facility': 'ASGA'},
  {'facility': 'ASGB'},
  {'facility': 'GFA'},
  {'facility': 'GFB'},
  {'facility': 'GFC'},
  {'facility': 'GKR'},
  {'facility': 'HD'},
  {'facility': 'JSV'},
  {'facility': 'KRIS'},
  {'facility': 'MARA'},
  {'facility': 'MARB'},
  {'facility': 'MLA'},
  {'facility': 'NOR'},
  {'facility': 'OSF'},
  {'facility': 'TROA'},
  {'facility': 'VIS'}]}

In [26]:
_http_client = HttpClient(
            azure_credential=credential, resource_id=resource_id)

In [57]:
_http_client.request(request_type='get', url=base_url+'/sources', accept='application/json', payload=None, params=None)#, headers=headers)

{'statusCode': 401,
 'message': 'Unauthorized. Access token is missing or invalid.'}

In [2]:
api = AEAPI(azure_credential=credential, environment=AEEnvironment.Dev())

NameError: name 'AEAPI' is not defined